In [1]:
import time
import tensorflow as tf
import numpy as np
from glob import glob
import datetime
import random
from PIL import Image
import matplotlib.pyplot as plt
%matplotlib inline
from  tensorflow import keras
from  tensorflow.keras import layers
import base64
import cv2
import os
from tqdm import tqdm
from tensorflow.keras.utils import img_to_array
import re
from imutils import paths
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from keras.models import Sequential
from keras.layers.core import Dense
from keras.optimizers import SGD
from keras.optimizers import Adam
from sklearn.metrics import classification_report
from keras.preprocessing.image import ImageDataGenerator

from keras.models import Sequential
from keras.layers import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers import LeakyReLU
from keras.layers.core import Flatten, Dropout

SEED=1337

In [2]:
# initialize the data and labels
print("[INFO] loading images...")
images = []
labels = []

# grab the image paths and randomly shuffle them
imagePaths = sorted(list(paths.list_images('train')))
random.seed(SEED)
random.shuffle(imagePaths)

# loop over the input images
for imagePath in imagePaths:
    # load the image, resize it to 64x64 pixels (the required input spatial dimensions of SmallVGGNet), 
    # and store the image in the data list
    image = cv2.imread(imagePath)
    image = cv2.resize(image, (224, 224))   # we are not flattening our data for neural network, because it is convolutional
    images.append(image)

    # extract the class label from the image path and update the labels list
    label = imagePath.split(os.path.sep)[1]
    labels.append(label)

# scale the raw pixel intensities to the range [0, 1]
images = np.array(images, dtype="float") / 255.0
labels = np.array(labels)

print('done')

[INFO] loading images...
done


In [3]:
# partition the data into 80% training and 20% validation
(trainX, valX, trainY, valY) = train_test_split(images, labels, test_size=0.2, random_state=SEED)

In [4]:
trainX.shape

(3584, 224, 224, 3)

In [5]:
lb = preprocessing.LabelBinarizer()
trainY = lb.fit_transform(trainY)
valY = lb.transform(valY)

In [6]:
input_shape = (trainX.shape[1], trainX.shape[2], trainX.shape[3])

In [11]:
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input

## Loading VGG16 model
base_model = VGG16(weights="imagenet", include_top=False, input_shape=input_shape)
base_model.trainable = False ## Not trainable weights

In [12]:
base_model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [13]:
from tensorflow.keras import layers, models

flatten_layer = layers.Flatten()
dense_layer_1 = layers.Dense(1024, activation='relu')
# dropput_layer_1 = layers.Dropout(0.25)
# dense_layer_2 = layers.Dense(512, activation='relu')
prediction_layer = layers.Dense(1, activation='sigmoid')


model = models.Sequential([
    base_model,
    flatten_layer,
    dense_layer_1,
#     dropput_layer_1,
#     dense_layer_2,
    prediction_layer
])

In [6]:
model = tf.keras.models.load_model('saved_model/vgg_test.h5')

In [14]:
aug = ImageDataGenerator(rotation_range=30, width_shift_range=0.1,
                         height_shift_range=0.1, shear_range=0.2, zoom_range=0.2,
                         horizontal_flip=True)

In [15]:
from tensorflow.keras.callbacks import EarlyStopping

INIT_LR = 1e-4
EPOCHS = 5
BS = 32

opt = SGD(INIT_LR)
model.compile(loss="binary_crossentropy", optimizer=opt, metrics=["binary_accuracy"])

model.fit(aug.flow(trainX, trainY, batch_size=16), epochs=EPOCHS, validation_data=(valX, valY), batch_size=16, 
          callbacks=[keras.callbacks.EarlyStopping(patience=11, verbose=1, restore_best_weights=True),
                    keras.callbacks.ReduceLROnPlateau(factor=.5, patience=4, verbose=1)])

Epoch 1/5
224/224 [==============================] - 27s 119ms/step - loss: 0.6782 - binary_accuracy: 0.5826 - val_loss: 0.6684 - val_binary_accuracy: 0.5960 - lr: 1.0000e-04
Epoch 2/5
224/224 [==============================] - 25s 113ms/step - loss: 0.6532 - binary_accuracy: 0.6102 - val_loss: 0.6484 - val_binary_accuracy: 0.6328 - lr: 1.0000e-04
Epoch 3/5
224/224 [==============================] - 25s 113ms/step - loss: 0.6354 - binary_accuracy: 0.6437 - val_loss: 0.6323 - val_binary_accuracy: 0.6484 - lr: 1.0000e-04
Epoch 4/5
224/224 [==============================] - 25s 112ms/step - loss: 0.6212 - binary_accuracy: 0.6682 - val_loss: 0.6187 - val_binary_accuracy: 0.6629 - lr: 1.0000e-04
Epoch 5/5
224/224 [==============================] - 25s 113ms/step - loss: 0.6056 - binary_accuracy: 0.6895 - val_loss: 0.6104 - val_binary_accuracy: 0.6663 - lr: 1.0000e-04


In [24]:
model.fit(aug.flow(trainX, trainY, batch_size=16), epochs=5, validation_data=(valX, valY), batch_size=16, 
          callbacks=[keras.callbacks.EarlyStopping(patience=11, verbose=1, restore_best_weights=True),
                    keras.callbacks.ReduceLROnPlateau(factor=.5, patience=4, verbose=1)])

Epoch 1/10
224/224 [==============================] - 29s 128ms/step - loss: 0.6551 - binary_accuracy: 0.6155 - val_loss: 0.6568 - val_binary_accuracy: 0.5949 - lr: 1.0000e-06
Epoch 2/10
224/224 [==============================] - 25s 112ms/step - loss: 0.6557 - binary_accuracy: 0.6141 - val_loss: 0.6566 - val_binary_accuracy: 0.5938 - lr: 1.0000e-06
Epoch 3/10
224/224 [==============================] - 25s 111ms/step - loss: 0.6528 - binary_accuracy: 0.6175 - val_loss: 0.6564 - val_binary_accuracy: 0.5949 - lr: 1.0000e-06
Epoch 4/10
224/224 [==============================] - 25s 114ms/step - loss: 0.6554 - binary_accuracy: 0.6180 - val_loss: 0.6562 - val_binary_accuracy: 0.5960 - lr: 1.0000e-06
Epoch 5/10
224/224 [==============================] - 25s 113ms/step - loss: 0.6520 - binary_accuracy: 0.6194 - val_loss: 0.6560 - val_binary_accuracy: 0.5960 - lr: 1.0000e-06
Epoch 6/10
224/224 [==============================] - 25s 112ms/step - loss: 0.6543 - binary_accuracy: 0.6189 - val_loss

In [16]:
model.save("saved_model/vgg_test.h5")

In [17]:
for layer in model.layers:
    layer.trainable = True

EPOCHS = 10

opt = Adam(1e-6)

model.compile(loss="binary_crossentropy", optimizer=opt, metrics=["binary_accuracy"])

model.fit(aug.flow(trainX, trainY, batch_size=16), epochs=EPOCHS, validation_data=(valX, valY), batch_size=16, 
          callbacks=[keras.callbacks.EarlyStopping(patience=11, verbose=1, restore_best_weights=True),
                    keras.callbacks.ReduceLROnPlateau(factor=.5, patience=4, verbose=1)])

Epoch 1/10
224/224 [==============================] - 40s 151ms/step - loss: 0.5580 - binary_accuracy: 0.7366 - val_loss: 0.5191 - val_binary_accuracy: 0.7679 - lr: 1.0000e-06
Epoch 2/10
224/224 [==============================] - 32s 143ms/step - loss: 0.4832 - binary_accuracy: 0.7826 - val_loss: 0.4451 - val_binary_accuracy: 0.8013 - lr: 1.0000e-06
Epoch 3/10
224/224 [==============================] - 32s 143ms/step - loss: 0.4061 - binary_accuracy: 0.8284 - val_loss: 0.4103 - val_binary_accuracy: 0.8080 - lr: 1.0000e-06
Epoch 4/10
224/224 [==============================] - 32s 143ms/step - loss: 0.3472 - binary_accuracy: 0.8527 - val_loss: 0.3456 - val_binary_accuracy: 0.8493 - lr: 1.0000e-06
Epoch 5/10
224/224 [==============================] - 32s 143ms/step - loss: 0.3026 - binary_accuracy: 0.8764 - val_loss: 0.2894 - val_binary_accuracy: 0.8817 - lr: 1.0000e-06
Epoch 6/10
224/224 [==============================] - 32s 144ms/step - loss: 0.2654 - binary_accuracy: 0.8901 - val_loss

In [18]:
model.save("saved_model/vgg_test.h5")

In [19]:
print("[INFO] loading test images...")
testData = []
testLabel = []

# grab the image paths and randomly shuffle them
testImagePaths = sorted(list(paths.list_images('test')))
random.seed(SEED)
random.shuffle(testImagePaths)

# loop over the input images
for imagePath in testImagePaths:
    # load the image, resize it to 64x64 pixels (the required input spatial dimensions of SmallVGGNet), 
    # and store the image in the data list
    image = cv2.imread(imagePath)
    image = cv2.resize(image, (224, 224))   # we are not flattening our data for neural network, because it is convolutional
    testData.append(image)

    # extract the class label from the image path and update the labels list
    label = imagePath.split(os.path.sep)[1]
    testLabel.append(label)

# scale the raw pixel intensities to the range [0, 1]
testData = np.array(testData, dtype="float") / 255.0
testLabel = np.array(testLabel)

print('done')

[INFO] loading test images...
done


In [3]:
testData.shape

(2046, 224, 224, 3)

In [20]:
testY = lb.transform(testLabel)

In [5]:
lb = preprocessing.LabelBinarizer()
testY = lb.fit_transform(testLabel)

### Evaluation

In [6]:
model = tf.keras.models.load_model('saved_model/vgg_test.h5')

In [21]:
predictions = model.predict(testData,  batch_size=8)

256/256 [==============================] - 7s 24ms/step


In [22]:
predictions =  (predictions>0.5)
print(classification_report(testY, predictions))

              precision    recall  f1-score   support

           0       0.77      0.75      0.76      1271
           1       0.61      0.63      0.62       775

    accuracy                           0.70      2046
   macro avg       0.69      0.69      0.69      2046
weighted avg       0.71      0.70      0.70      2046



In [24]:
model.save("vgg_test_accuracy_70.h5")